# DeBERTa LLRD + LastLayerReinit with TensorFlow

It Uses:

* MultilabelStratifiedKFold split of the data
* HuggingFace DeBERTaV3 pre-trained model finetuning with Tensorflow
* WeightedLayerPool + MeanPool TensorFlow implementation
* Layer-wise learning rate decay
* Last layer reinitialization or partially reinitialzation

# Imports

In [ ]:
import os, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print(f'TF version: {tf.__version__}')
import tensorflow_addons as tfa
from tensorflow.keras import layers

import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
#   os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)

# Load DataFrame

In [ ]:
df = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
display(df.head())
print('\n---------DataFrame Summary---------')
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px #graphing
import plotly.graph_objects as go #graphing
from plotly.subplots import make_subplots #graphing
import plotly.figure_factory as ff #graphing

In [ ]:
colormap = sns.color_palette('Blues')
sns.heatmap(df.corr(), annot = True, cmap = colormap )

In [ ]:
fig = px.histogram(df, x = df['full_text'].str.len(), marginal = 'box', title  = 'Histogram of full_text text length', 
                  color_discrete_sequence = ["#FFA200"] )

fig.update_layout(bargap = 0.2)

fig.update_layout(template = 'plotly_dark', font = dict(family = 'PT Sans', size = 19, color = "#C4FEFF"  ) )

fig.show()

In [ ]:
colors = [["#00E600"], ["#0000E6"], ["#E600DF"], ["#E6E600"], ["#FFFFFF"], ["#cd040b"]]


for count, x in enumerate(["cohesion", "syntax", "vocabulary", "phraseology", "grammar", "conventions"]):
    fig = px.histogram(df, x = x, marginal = 'violin', title = f" {x} histogram", color_discrete_sequence = colors[count]  )
    
    fig.update_layout(bargap = 0.2)
    fig.update_layout(template = 'plotly_dark', font = dict(family = 'PT Sans', size = 19, color = "#C4FEFF" ) )
    
    fig.show()

# CV Split

In [ ]:
N_FOLD = 5
TARGET_COLS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

skf = MultilabelStratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(skf.split(df, df[TARGET_COLS])):
    df.loc[val_index, 'fold'] = int(n)
df['fold'] = df['fold'].astype(int)
df['fold'].value_counts()

In [ ]:
df.to_csv('./df_folds.csv', index=False)

# Model Config

In [ ]:
MAX_LENGTH = 512
BATCH_SIZE = 14

DEBERTA_MODEL = "../input/debertav3base"

Why we should disable dropout in regression task, check this [discussion](https://www.kaggle.com/competitions/commonlitreadabilityprize/discussion/260729).

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(DEBERTA_MODEL)
tokenizer.save_pretrained('./tokenizer/')

cfg = transformers.AutoConfig.from_pretrained(DEBERTA_MODEL, output_hidden_states=True)
cfg.hidden_dropout_prob = 0
cfg.attention_probs_dropout_prob = 0
cfg.save_pretrained('./tokenizer/')

# Data Process Function

To make use of HugggingFace DeBERTa model, we have to tokenize our input texts as the pretrained DeBERTa model requires.

In [ ]:
def deberta_encode(texts, tokenizer=tokenizer):
    input_ids = []
    attention_mask = []
    for text in texts.tolist():
        token = tokenizer(text, 
                          add_special_tokens=True, 
                          max_length=MAX_LENGTH, 
                          return_attention_mask=True, 
                          return_tensors="np", 
                          truncation=True, 
                          padding='max_length')
        input_ids.append(token['input_ids'][0])
        attention_mask.append(token['attention_mask'][0])
    
    return np.array(input_ids, dtype="int32"), np.array(attention_mask, dtype="int32")

In [ ]:
def get_dataset(df):
    inputs = deberta_encode(df['full_text'])
    targets = np.array(df[TARGET_COLS], dtype="float32")
    return inputs, targets

# Model

## MeanPool

Instead of using '[CLS]' token, MeanPool method averaging one layer of hidden states along the sequence axis with masking out padding tokens.

## WeightedLayerPool

WeightedLayerPool uses a set of trainable weights to average a set of hidden states from transformer backbone. I use a Dense layer with constraint to implement it.

In [ ]:
class MeanPool(tf.keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        embedding_sum = tf.reduce_sum(inputs * broadcast_mask, axis=1)
        mask_sum = tf.reduce_sum(broadcast_mask, axis=1)
        mask_sum = tf.math.maximum(mask_sum, tf.constant([1e-9]))
        return embedding_sum / mask_sum

WeightedLayerPool weights constraints: softmax to push sum(w) to be 1.

In [ ]:
class WeightsSumOne(tf.keras.constraints.Constraint):
    def __call__(self, w):
        return tf.nn.softmax(w, axis=0)

## Model Design Choice

Although there are many ways to get your final representations, I choose to take the last 4 layers hidden states of DeBERTa, take MeanPool of them to gather information along the sequence axis, then take WeightedLayerPool with a set of trainable weights to gather information along the depth axis of the model, then finally a regression head.

## Last Layer Reinitialization

Reinitialization of the last transformer encoder block: GlorotUniform for Dense kernel, Zeros for Dense bias, Zeros for LayerNorm beta, Ones for LayerNorm gamma.

## Layer-wise Learning Rate Decay

I use MultiOptimizer to implement LLRD: Initial learning rate 1e-5 with layer-wise decay 0.9 for transformer encoder and embedding block, 1e-4 for the rest of the model. All learning rates have ExponentialDecay schedulers with decay rate 0.3

In [ ]:
def get_model():
    input_ids = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="input_ids"
    )
    
    attention_masks = tf.keras.layers.Input(
        shape=(MAX_LENGTH,), dtype=tf.int32, name="attention_masks"
    )
   
    deberta_model = transformers.TFAutoModel.from_pretrained(DEBERTA_MODEL, config=cfg)
    
    #Last Layer Reinitialization or Partially Reinitialization
#     Uncommon next three lines to check deberta encoder block
#     print('DeBERTa Encoder Block:')
#     for layer in deberta_model.deberta.encoder.layer:
#         print(layer)
        
    REINIT_LAYERS = 1
    normal_initializer = tf.keras.initializers.GlorotUniform()
    zeros_initializer = tf.keras.initializers.Zeros()
    ones_initializer = tf.keras.initializers.Ones()

#     print(f'\nRe-initializing encoder block:')
    for encoder_block in deberta_model.deberta.encoder.layer[-REINIT_LAYERS:]:
#         print(f'{encoder_block}')
        for layer in encoder_block.submodules:
            if isinstance(layer, tf.keras.layers.Dense):
                layer.kernel.assign(normal_initializer(shape=layer.kernel.shape, dtype=layer.kernel.dtype))
                if layer.bias is not None:
                    layer.bias.assign(zeros_initializer(shape=layer.bias.shape, dtype=layer.bias.dtype))

            elif isinstance(layer, tf.keras.layers.LayerNormalization):
                layer.beta.assign(zeros_initializer(shape=layer.beta.shape, dtype=layer.beta.dtype))
                layer.gamma.assign(ones_initializer(shape=layer.gamma.shape, dtype=layer.gamma.dtype))

    deberta_output = deberta_model.deberta(
        input_ids, attention_mask=attention_masks
    )
    hidden_states = deberta_output.hidden_states
    
    #WeightedLayerPool + MeanPool of the last 4 hidden states
    stack_meanpool = tf.stack(
        [MeanPool()(hidden_s, mask=attention_masks) for hidden_s in hidden_states[-4:]], 
        axis=2)
    
    weighted_layer_pool = layers.Dense(1,
                                       use_bias=False,
                                       kernel_constraint=WeightsSumOne())(stack_meanpool)
    
    weighted_layer_pool = tf.squeeze(weighted_layer_pool, axis=-1)
    
    x = layers.Dense(6, activation='sigmoid')(weighted_layer_pool)
    output = layers.Rescaling(scale=4.0, offset=1.0)(x)
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)
    
    #Compile model with Layer-wise Learning Rate Decay
    layer_list = [deberta_model.deberta.embeddings] + list(deberta_model.deberta.encoder.layer)
    layer_list.reverse()
    
    INIT_LR = 1e-5
    LLRDR = 0.9
    LR_SCH_DECAY_STEPS = 1600 # 2 * len(train_df) // BATCH_SIZE
    
    lr_schedules = [tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=INIT_LR * LLRDR ** i, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3) for i in range(len(layer_list))]
    lr_schedule_head = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4, 
        decay_steps=LR_SCH_DECAY_STEPS, 
        decay_rate=0.3)
    
    optimizers = [tf.keras.optimizers.Adam(learning_rate=lr_sch) for lr_sch in lr_schedules]
    
    optimizers_and_layers = [(tf.keras.optimizers.Adam(learning_rate=lr_schedule_head), model.layers[-4:])] +\
        list(zip(optimizers, layer_list))
    
#     Uncomment next three lines to check optimizers_and_layers
#     print('\nLayer-wise Learning Rate Decay Initial LR:')
#     for o,l in optimizers_and_layers:
#         print(f'{o._decayed_lr("float32").numpy()} for {l}')
        
    optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)
    
    model.compile(optimizer=optimizer,
                 loss='huber_loss',
                 metrics=[tf.keras.metrics.RootMeanSquaredError()],
                 )
    return model

In [ ]:
# configure gpu
gpus = tf.config.list_physical_devices('GPU'); 
if len(gpus) == 1:
    strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.OneDeviceStrategy(device='/cpu:0')

In [ ]:
# ENABLE MIXED PRECISION for speed
#tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')



In [ ]:
tf.keras.backend.clear_session()
with strategy.scope():
    model = get_model()
model.summary()


# 5 Folds Training Loop

In [ ]:
valid_rmses = []
for fold in range(N_FOLD):
    print(f'\n-----------FOLD {fold} ------------')
    
    #Create dataset
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    valid_df = df[df['fold'] == fold].reset_index(drop=True)
    train_dataset = get_dataset(train_df)
    valid_dataset = get_dataset(valid_df)
    print('Data prepared.')
    
    #Create model
    tf.keras.backend.clear_session()
    with strategy.scope():
        model = get_model()
    print('Model prepared.')
    
    #Training model
    print('Start training...')
    callbacks = [
    tf.keras.callbacks.ModelCheckpoint(f"best_model_fold{fold}.h5",
                                       monitor="val_loss",
                                       mode="min",
                                       save_best_only=True,
                                       verbose=1,
                                       save_weights_only=True,),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                     min_delta=1e-5, 
                                     patience=3, 
                                     verbose=1,
                                     mode='min',)
    ]
    history = model.fit(x=train_dataset[0],
                        y=train_dataset[1],
                        validation_data=valid_dataset, 
                        epochs=10,
                        shuffle=True,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        callbacks=callbacks
                       )
    
    valid_rmses.append(np.min(history.history['val_root_mean_squared_error']))
    print('Training finished.')
    del train_dataset, valid_dataset, train_df, valid_df
    gc.collect()

In [ ]:
print(f'{len(valid_rmses)} Folds validation RMSE:\n{valid_rmses}')
print(f'Local CV Average score: {np.mean(valid_rmses)}')

# Inference and Submission



In [ ]:
test_df = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
test_df.head()

In [ ]:
test_dataset = deberta_encode(test_df['full_text'])

# 5 Folds ensemble prediction

In [ ]:
fold_preds = []
for fold in range(N_FOLD):
    tf.keras.backend.clear_session()
    model = get_model()
    model.load_weights(f'best_model_fold{fold}.h5')
    print(f'\nFold {fold} inference...')
    pred = model.predict(test_dataset, batch_size=BATCH_SIZE)
    fold_preds.append(pred)
    gc.collect()

In [ ]:
preds = np.mean(fold_preds, axis=0)
preds = np.clip(preds, 1, 5)
preds = np.round(2*preds)/2
preds

In [ ]:
sub_df = pd.concat([test_df[['text_id']], pd.DataFrame(preds, columns=TARGET_COLS)], axis=1)
sub_df.to_csv('submission.csv', index=False)

In [ ]:
sub_df.head()